Upgrade python package `requests`.

In [ ]:
!pip install google-cloud requests keras tensorflow

Import python packages.

In [ ]:
# Imports the Google Cloud client library
from google.cloud import storage
from google.oauth2 import service_account

from google.cloud.exceptions import NotFound

# Inline display image
from IPython.display import Image, display

# Imports Keras packages and model
import numpy as np
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input, decode_predictions


Update global setting to fit into your private environments.

In [ ]:
# Credential JSON file
CREDENTIALS_JSON='./datalab-trial-71072497752e.json'
# GCP project id
PROJECT_ID='datalab-trial-186403'
# Bucket name of Cloud Storage
GCS_BUCKET_NAME='game_ml_training_pics_10k'

In [ ]:
# Credentials for GCP
credentials = service_account.Credentials.from_service_account_file(CREDENTIALS_JSON)

# Instantiates a cloud storage client
storage_client = storage.Client(project=PROJECT_ID, credentials=credentials)

def show_blob(blob):
    blob.download_to_filename('/tmp/blob.jpeg')
    display(Image(filename='/tmp/blob.jpeg'))

    
def predict(blob):
    model = InceptionV3(weights='imagenet')

    # Make input data from Jpeg file
    img_path = '/tmp/blob.jpeg'
    blob.download_to_filename(img_path)
    img = image.load_img(img_path, target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    # Classify image
    preds = model.predict(x)

    # Print predicted classes
    print('Predicted:')
    for p in decode_predictions(preds, top=5)[0]:
        print("Score {}, Label {}".format(p[2], p[1]))    


if __name__ == '__main__':
    # check bucket availability
    try:
        bucket = storage_client.get_bucket(GCS_BUCKET_NAME)
    except NotFound:
        print('Sorry, the bucket {} does not exist!'.format(GCS_BUCKET_NAME))
    
    # iterate files in the bucket
    for f in bucket.list_blobs():
        show_blob(f)
        predict(f)
        break

        